In [10]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from embetter.grab import ColumnGrabber
from embetter.multi import ClipEncoder
from embetter.text import SentenceEncoder
from embetter.vision import ImageLoader
from sklearn.linear_model import (LogisticRegression, Perceptron,
                                  QuantileRegressor)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

# Rury

In [20]:
rura_img = make_pipeline(
    ImageLoader(convert="RGB"),
    ClipEncoder(),
)

rura_text = make_pipeline(
    ColumnGrabber("text"),
    SentenceEncoder()
)
rura_text2 = make_pipeline(
    ColumnGrabber("Text"),
    SentenceEncoder()
)


# Dane

In [21]:
paths_pets = []
labels_pets = []

paths_cats = []
labels_cats = []



for path in Path("pets").glob("*.jpg"):
    stem = path.stem
    paths_pets.append(str(path))
    labels_pets.append(stem[:stem.rfind("_")])

for path in Path("kotki").iterdir():
    for sub_path in Path(path).glob('*.jpg'):
        labels_cats.append(sub_path.parent.name)
        paths_cats.append(str(sub_path))


#DLACZEGO DATASETY NA KAGGLE'U SA ZJEBANE?

df = pd.read_csv('twitter_training.csv')
df.columns = ['id', 'entity', 'sentiment', 'text']
df.dropna(inplace=True)

df['entity'] = df['entity'].astype('string')
df['sentiment'] = df['sentiment'].astype('string')
df['text'] = df['text'].astype('string')

df2 = pd.read_csv('dataset.csv')
df2.dropna(inplace=True)

0        klement gottwaldi surnukeha palsameeriti ning ...
1        sebes joseph pereira thomas  på eng the jesuit...
2        ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...
3        விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...
4        de spons behoort tot het geslacht haliclona en...
                               ...                        
21995    hors du terrain les années  et  sont des année...
21996    ใน พศ  หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...
21997    con motivo de la celebración del septuagésimoq...
21998    年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...
21999     aprilie sonda spațială messenger a nasa și-a ...
Name: Text, Length: 22000, dtype: object

# Modele

In [13]:
X_1 = rura_img.transform(paths_pets)
X_2 = rura_img.transform(paths_cats)

X_3 = rura_text.transform(df[:len(df)//2])


Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.
Unused or unrecognized kwargs: padding.


In [23]:
X_4 = rura_text2.transform(df2)

In [14]:
X_1

array([[-0.03038803, -0.4069671 ,  0.22537117, ...,  0.30013418,
        -0.49421257, -0.3539784 ],
       [ 0.22172228,  0.10130153,  0.27347124, ...,  0.76611614,
        -0.3093354 , -0.04325171],
       [ 0.1225246 , -0.332682  ,  0.12871505, ...,  0.13546167,
        -0.44622442, -0.33025128],
       ...,
       [-0.01610568, -0.24678226, -0.14172104, ...,  0.8317056 ,
         0.09846329, -0.07646413],
       [ 0.2540474 ,  0.13698183, -0.06307959, ...,  0.7262149 ,
         0.10655477,  0.18177734],
       [-0.04472077,  0.16235577, -0.17671788, ...,  0.89453113,
         0.2877938 , -0.03816501]], dtype=float32)

In [15]:
model_1 = LogisticRegression(max_iter=1000)
model_2 = Perceptron()
model_3 = QuantileRegressor()

# Wyniki

In [25]:
means_1 = [np.mean(cross_val_score(model_1, X_1, labels_pets, cv=5)), 
         np.mean(cross_val_score(model_2, X_1, labels_pets, cv=5)), 
        #  np.mean(cross_val_score(model_3, X_1, labels_pets, cv=5))
         ]


means_2 = [np.mean(cross_val_score(model_1, X_2, labels_cats, cv=5)), 
         np.mean(cross_val_score(model_2, X_2, labels_cats, cv=5)), 
        #  np.mean(cross_val_score(model_3, X_2, labels_cats, cv=5))
         ]

means_3 = [np.mean(cross_val_score(model_1, X_3, df[:len(df)//2]['sentiment'], cv=5)), 
         np.mean(cross_val_score(model_2, X_3, df[:len(df)//2]['sentiment'], cv=5)), 
        #  np.mean(cross_val_score(model_3, X_2, labels_cats, cv=5))
         ]

means_4 = [np.mean(cross_val_score(model_1, X_4, df2['language'], cv=5)), 
         np.mean(cross_val_score(model_2, X_4, df2['language'], cv=5)), 
        #  np.mean(cross_val_score(model_3, X_2, labels_cats, cv=5))
         ]


(means_1, means_2, means_3, means_4)

([0.86, 0.842],
 [0.861277072442121, 0.8381067961165047],
 [0.5009589937281517, 0.38957066879015795],
 [0.9644999999999999, 0.9543636363636365])